In [1]:
# project.py
#from https://projects.raspberrypi.org/en/projects/get-started-pico-w/

%serialconnect /dev/ttyACM0
%connect /dev/ttyACM0

import network
import socket
from time import sleep
from picozero import pico_temp_sensor, pico_led
import machine
import rp2
import sys
import secrets

No device connected
  %serialconnect or websocketconnect to connect
  %lsmagic to list commands

## Resources Usefull in  Programming the Pico
1. [Vim key bindings](https://vimsheet.com)
1. [Markdown Cheatsheet](https://www.markdownguide.org/cheat-sheet/)
2. [Top level Jupyter Docs](https://docs.jupyter.org/en/latest/)
3. [Raspberry Pico -- Getting Started](https://projects.raspberrypi.org/en/projects/get-started-pico-w/0)
4. [pico-sdk/examples_page.html](https://www.raspberrypi.com/documentation/pico-sdk/examples_page.html)
5. [Pico-micropython-examples](https://github.com/raspberrypi/pico-micropython-examples)
6. [Debug python ernel](https://github.com/goatchurchprime/jupyter_micropython_kernel)
1. [Oficcial SDK](https://github.com/raspberrypi/pico-sdk)
2. [Github package for Kernel that works with Jupyter__Good README](https://github.com/Carglglz/jupyter_upydevice_kernel) 
## History
1. Why does my Laptop Fail to connect to an open Socket?
   a. The fact that the Pico has a radio that functions only on 2.4 gHz may trip you up.

In [1]:
import machine

No device connected
  %serialconnect or websocketconnect to connect
  %lsmagic to list commands

In [ ]:
def connect():
    #Connect to WLAN
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    wlan.connect(secrets.WIFI_SSID, secrets.WIFI_PASSWORD)
    while wlan.isconnected() == False:
        if rp2.bootsel_button() == 1:
            sys.exit()
        print('Waiting for connection...')
        pico_led.on()
        sleep(0.5)
        pico_led.off()
        sleep(0.5)

    ip = wlan.ifconfig()[0]
    print(wlan.ifconfig())
    print(f'Connected on {ip}')
    ip = wlan.ifconfig()[0]
    print(f'Connected on {ip}')
    return(ip)

def open_socket(ip):
    # Open a socket
    address = (ip, 80)
    connection = socket.socket()
    connection.bind(address)
    connection.listen(1)
    print(connection)
    return connection

def webpage(temperature, state):
    #Template HTML
    html = f"""
            <!DOCTYPE html>
            <html>
            <form action="./lighton">
            <input type="submit" value="Light on" />
            </form>
            <form action="./lightoff">
            <input type="submit" value="Light off" />
            </form>
            <p>LED is {state}</p>
            <p>Temperature is {temperature}</p>
            </body>
            </html>
            """
    return str(html)

def serve(connection):
    #Start a web server
    state = 'OFF'
    pico_led.off()
    temperature = 0
    while True:
        client = connection.accept()[0]
        request = client.recv(1024)
        request = str(request)
        #print(request)
        try:
            request = request.split()[1]
        except IndexError:
            pass
        if request == '/lighton?':
            pico_led.on()
            state = 'ON'
        elif request =='/lightoff?':
            pico_led.off()
            state = 'OFF'
        elif request == '/close?':
            sys.exit()
        temperature = pico_temp_sensor.temp
        html = webpage(temperature, state)

        html = webpage(temperature, state)
        client.send(html)
        client.close()


ip = connect()
connection = open_socket(ip)
serve(connection)

In [3]:
from rp2 import machine 
rtc = machine.RTC()
#rtc.datetime((2025, 11, 22, 15, 57, 32, 36, 0))
print(rtc.datetime())

No device connected
  %serialconnect or websocketconnect to connect
  %lsmagic to list commands

In [1]:
import network
import rp2

# Ensure your country code is configured
rp2.country('US')

# Create a Wi-Fi station interface
station = network.WLAN(network.STA_IF)
station.active(True)

# Scan for networks
print("Scanning for Wi-Fi networks...")
networks = station.scan()

print(f"Found {len(networks)} networks:")
for ssid, bssid, channel, RSSI, authmode, hidden in networks:
    print(f"  SSID: {ssid.decode()}, RSSI: {RSSI}, Auth: {authmode}")

station.active(False)


No device connected
  %serialconnect or websocketconnect to connect
  %lsmagic to list commands

In [2]:
%lsmagic

%disconnect
    disconnects device

%lsmagic
    list magic commands

%rebootdevice
    reboots device

%is_reachable
    Test if device is reachable (must be connected first)

%connect
    connects to a device based on addres or configuration

%serialconnect [-kbi] [portname] [baudrate]
    connects to a device over USB, default baudrate is 115200

%websocketconnect [--password PASSWORD] [-kbi] [-ssl] [websocketurl]
    connects to the WebREPL over wifi (WebREPL daemon must be running)
    websocketurl defaults to 192.168.4.1 (uri -> ws://192.168.4.1:8266)

%bleconnect [bleaddress]
    connects to the BleREPL over Bluetooth Low Energy(BleREPL must be running)

%meminfo
    Shows RAM size/used/free/use% info

%whoami
    Shows Device name, port, id, and system info

%rssi
    Shows Device RSSI if wireless

%gccollect
    To use the garbage collector and free some RAM if possible

%local
    To run the cell contents in local iPython

%sync
    To sync a variable/output data structure of

In [4]:
%connect /dev/ttyACM0




*** Sending Ctrl-C



In [2]:
# SD CARD -- drive picowbell sd pico hat by Adafruit - $8
def sd_init():
    # SPDX-FileCopyrightText: 2023 Liz Clark for Adafruit Industries
    #
    # SPDX-License-Identifier: MIT
    """CircuitPython PiCowbell Adalogger Example"""
    import time
    import machine
    import sdcardio
    import busio
    import storage
    
    #  setup for Pico I2C
    i2c = busio.I2C(board.GP5, board.GP4)
    
    # SPI SD_CS pin
    SD_CS = board.GP17
    
    #  SPI setup for SD card
    spi = busio.SPI(board.GP18, board.GP19, board.GP16)
    sdcard = sdcardio.SDCard(spi, SD_CS)
    vfs = storage.VfsFat(sdcard)
    try:
        storage.mount(vfs, "/sd")
        print("sd card mounted")
    except ValueError:
        print("no SD card")
#if __name__ == "__main__":
sd_init()

ModuleNotFoundError: No module named 'machine'

In [1]:
import board

ModuleNotFoundError: No module named 'board'

In [1]:
import serial
import time

# Configure the serial connection (adjust baud rate as needed for your device)
# Replace '/dev/ttyACM0' if your device appeared as a different port
try:
    ser = serial.Serial('/dev/ttyACM0', 9600, timeout=1)
    time.sleep(2) # Wait for the connection to establish
    print(f"Connected to {ser.name}")
except serial.SerialException as e:
    print(f"Error opening serial port: {e}")
    print("Please check the port name and permissions.")

# Example: Write data
if ser and ser.is_open:
    ser.write(b'Hello Device\n') # Write a byte string
    print("Data sent")

    # Example: Read data
    while ser.in_waiting:
        line = ser.readline().decode('utf-8').strip()
        print(f"Received: {line}")

    # Close the port when done
    ser.close()
    print("Port closed")


No device connected
  %serialconnect or websocketconnect to connect
  %lsmagic to list commands

In [3]:
%%bash
# This runs the following lines in a shell
ls -l /dev/ttyACM0
sudo dmesg | tail


crw-rw---- 1 root dialout 166, 0 Dec  1 15:31 /dev/ttyACM0
[   38.752565] usb 3-1: Product: Board in FS mode
[   38.752567] usb 3-1: Manufacturer: MicroPython
[   38.752568] usb 3-1: SerialNumber: e661640843581e31
[   38.801760] cdc_acm 3-1:1.0: ttyACM0: USB ACM device
[   38.801799] usbcore: registered new interface driver cdc_acm
[   38.801801] cdc_acm: USB Abstract Control Model driver for USB modems and ISDN adapters
[  354.819159] logitech-hidpp-device 0003:046D:4091.0004: hidpp_root_get_protocol_version: received protocol error 0x08
[  457.737211] logitech-hidpp-device 0003:046D:4023.0003: hidpp_root_get_protocol_version: received protocol error 0x08
[  712.509175] warning: `ThreadPoolForeg' uses wireless extensions which will stop working for Wi-Fi 7 hardware; use nl80211
[ 1043.211603] logitech-hidpp-device 0003:046D:4023.0003: hidpp_root_get_protocol_version: received protocol error 0x08


In [1]:
print('hi')

hi
